In [2]:
import os
from dotenv import load_dotenv

platform_domain = "https://staging.eodatahub.org.uk"
workspace = "sgillies-tpzuk"
load_dotenv("demo.env")
token = os.environ["API_TOKEN"]
workspaces_bucket = "workspaces-eodhp-staging"

## Resource Catalogue

### Searching the entire catalogue

In [ ]:
!pip install pystac-client

In [ ]:
from pystac_client import Client

# Set resource catalogue top-level url
rc_url = f"{platform_domain}/api/catalogue/stac"

# We can also view the STAC Catalogue using STAC Browser here 
# https://staging.eodatahub.org.uk/static-apps/stac-browser/main/index.html#/external/staging.eodatahub.org.uk/api/catalogue/stac/

# Create STAC client
stac_client = Client.open(rc_url)

In [ ]:
# See all the available children in this catalogue
children = stac_client.get_children()

for child in children:
    print(child.id)

In [ ]:
# See all collections available (recursive searching)
collections = stac_client.get_collections()

for collection in collections:
    for link in collection.links:
        if link.rel == "self":
            print(f"Collection {collection.id} can be found at {link.href}")

In [ ]:
# Do some collection searching using keywords
collection_search = stac_client.collection_search(
    q="climate",
)
print(f"{collection_search.matched()} collections found")
for collection in collection_search.collections():
    print(collection.id)

In [ ]:
# Do some collection searching using bbox
uk_bbox = [10.854492,49.823809,100.762709,60.860699]
collection_search = stac_client.collection_search(
    bbox=uk_bbox,
)
print(f"{collection_search.matched()} collections found")
for collection in collection_search.collections():
    print(collection.id)

In [ ]:
# Do some collection searching using datetime, looking into the future
time_to_search = "2026-01-06T00:00:00Z/.."
collection_search = stac_client.collection_search(
    datetime=time_to_search,
)
print(f"{collection_search.matched()} collections found")
for collection in collection_search.collections():
    print(collection.id)

In [ ]:
# We can then get all the items in the cmip6 collection
print(collection.id)
items = collection.get_items()
print(next(collection.get_items(), None))

In [ ]:
# Do some item searching
# Geometry for the UK
geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-4.41191386337143, 50.5437323318846],
            [-2.86223024423239, 50.5522106222613],
            [-2.86502949737798, 49.564676297854],
            [-4.3832372671439, 49.5564878434316],
            [-4.41191386337143, 50.5437323318846]
        ]
    ],
}
search = stac_client.search(
    max_items=10,
    collections=['cmip6'],
    intersects=geom,
)
for item in search.items():
    print(item.id)

In [ ]:
# We can then see the assets for one of these items
assets = item.get_assets()

for asset in assets:
    print(f"Asset {asset} is available at {assets[asset].href}")

In [17]:
# Demonstrate that we support more complex searching via filters extension
filter = {
        "op": "and",
        "args": [
          {
            "op": "between",
            "args": [
              {
                "property": "properties.datetime"
              },
              "2020-01-12T00:00:00.000Z",
              "2025-02-12T23:59:59.999Z"
            ]
          },
          {
            "op": "and",
            "args": [
              {
                "op": "=",
                "args": [
                  {
                    "property": "collection"
                  },
                  "sentinel2_ard"
                ]
              },
              {
                "op": "<=",
                "args": [
                  {
                    "property": "properties.eo:cloud_cover"
                  },
                  5
                ]
              }
            ]
          }
        ]
      }
    
search = stac_client.search(
    max_items=10,
    filter=filter
)

for item in search.items():
    print(item.id)
    print(f"Datetime is {item.properties['datetime']}")
    print(f"Cloud cover is {item.properties['eo:cloud_cover']}")

# To then use such an item in a workflow, you can retrieve the href
for link in item.links:
    if link.rel == "self":
        print(f"Self href for item {item.id} is {link.href}")

NameError: name 'item' is not defined

### Our deployment also supports nested catalogs, which we can return by listing the children for a specific catalog

In [18]:
# Return children for a given nested catalog
from pystac_client import Client

# Set resource catalogue top-level url
rc_url = f"{platform_domain}/api/catalogue/stac/catalogs/supported-datasets"

# We can also view the STAC Catalogue using STAC Browser here 
# https://staging.eodatahub.org.uk/static-apps/stac-browser/main/index.html#/external/staging.eodatahub.org.uk/api/catalogue/stac/catalogs/supported-datasets

# Create STAC client
stac_client = Client.open(rc_url)

In [19]:
# See all the available children in this catalogue
children = stac_client.get_children()

for child in children:
    print(child.id)
    # We will also print the first three collections in each
    print(next(child.get_children()).id)

airbus
airbus_phr_data
ceda-stac-catalogue
cci
planet
PSScene
shahid-catalogue
shahid-collection


In [20]:
# Then we can see the child collections for one of these catalogs
# See all the available children in this catalogue
children = stac_client.get_children()

for child in children:
    if child.id == "ceda-stac-catalogue":
        ceda_catalog = child
        break

collections = ceda_catalog.get_children()

for collection in collections:
    print(collection.id)

cci
cmip6
cordex
eocis-aerosol-slstr-daily-s3a
eocis-aerosol-slstr-daily-s3b
eocis-aerosol-slstr-monthly-s3a
eocis-aerosol-slstr-monthly-s3b
eocis-lst-day
eocis-lst-s3a-day
eocis-lst-s3a-night
eocis-lst-s3b-day
eocis-lst-s3b-night
eocis-sst-cdrv3
eocis-sst-cdrv3-climatology
land_cover
sentinel1
sentinel2_ard
ukcp


### We can also access private catalogs by providing credentials to the client

In [30]:
# Return children for a given nested catalog
from pystac_client import Client

# Set resource catalogue top-level url
rc_url = f"{platform_domain}/api/catalogue/stac/catalogs/user-datasets"

# We can also view the STAC Catalogue using STAC Browser here 
# https://staging.eodatahub.org.uk/static-apps/stac-browser/main/index.html#/external/staging.eodatahub.org.uk/api/catalogue/stac/catalogs/user-datasets

# Create STAC client without authentication
stac_client = Client.open(rc_url)

In [31]:
# See all the available children in this catalogue
children = stac_client.get_children()

# We can then see that no child catalogs are provided, as auth is required for user catalogs

for child in children:
    print(child.id)


In [32]:
# And now with authentication
# Create STAC client
stac_client = Client.open(rc_url, headers={"Authorization": f"Bearer {token}"})

# See all the available children in this catalogue
children = stac_client.get_children()

# We can then see the child catalogs

for child in children:
    print(child.id)

sgillies-tpzuk


In [33]:
# Navigate to workflow output sub catalog
# And now with authentication
# Create STAC client
stac_client = Client.open(rc_url, headers={"Authorization": f"Bearer {token}"})
# See all the available children in this catalogue
children = stac_client.get_children()
# We can then see the child catalogs
for child in children:
    if child.id == workspace:
        priv_cat = child
        break
priv_cat_children = priv_cat.get_children()
for cat in priv_cat_children:
    if cat.id == "processing-results":
        proc_res_cat = cat
for workflow_cat in proc_res_cat.get_children():
    if workflow_cat.id == "convert-url-test":
        for job in workflow_cat.get_children():
            print(job.id)

cat_1277254e-e950-11ef-b749-ca3e8cd28499
cat_3f4db7ca-ed08-11ef-b95b-ca3e8cd28499


In [35]:
# Print output collection from workflow

collections = job.get_collections()

collection = next(collections)
items = collection.get_items()

### We also include a proxy for searching the Planet API

In [28]:
# Return children for a given nested catalog
from pystac_client import Client

# Set resource catalogue top-level url
rc_url = f"{platform_domain}/api/catalogue/stac/catalogs/supported-datasets/catalogs/planet"

# Create STAC client without authentication
stac_client = Client.open(rc_url)

In [29]:
# Now search for some items
collections = stac_client.get_children()

for collection in collections:
    print(collection.id)

PSScene
REOrthoTile
REScene
SkySatCollect
SkySatScene
SkySatVideo
TanagerMethane
TanagerScene


## Data Adaptors

This will start an Airbus SAR order

In [37]:
import requests

url = f"{platform_domain}/api/catalogue/manage/catalogs/user-datasets/{workspace}/commercial-data"
headers = {
    "accept": "application/json", 
    "Content-Type": "application/json", 
    "Authorization": f"Bearer {token}"
}
data =  {
    "url": f"{platform_domain}/api/catalogue/stac/catalogs/supported-datasets/catalogs/airbus/collections/airbus_sar_data/items/TSX-1_WS_S_wide_001R_97985_D33003943_29000",  
    "product_bundle": "general_use"
}

response = requests.post(url, headers=headers, json=data)

print("Status Code", response.status_code)
print("Response ", response.json())

Status Code 200
Response  {'message': 'Item ordered successfully'}


This will start an Airbus PHR order

In [38]:
import requests

url = f'{platform_domain}/api/catalogue/manage/catalogs/user-datasets/{workspace}/commercial-data'
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {token}'
}
data =  {
    "url": "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/supported-datasets/catalogs/airbus/collections/airbus_phr_data/items/DS_PHR1A_201203021558128_FR1_PX_W080S03_0221_01728",  
    "product_bundle": "general_use", 
    "coordinates": [
        [
            [-79.8,-2.1], 
            [-79.8,-2.2], 
            [-79.95,-2.2], 
            [-79.95,-2.1], 
            [-79.8,-2.1]
        ]
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code", response.status_code)
print("Response ", response.json())

Status Code 200
Response  {'message': 'Item ordered successfully'}


This will start a Planet order

In [3]:
import requests

url = f'{platform_domain}/api/catalogue/manage/catalogs/user-datasets/{workspace}/commercial-data'
headers = {
    'accept': 'application/json', 
    'Content-Type': 'application/json', 
    'Authorization': f'Bearer {token}'
}
data =  {
    "url": "https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/supported-datasets/catalogs/planet/collections/PSScene/items/20250217_101155_07_24c7",  
    "product_bundle": "analytic_udm2", 
    "coordinates": [
        [
            [9.6, 57.1],
            [9.6, 57.0],
            [9.8, 56.9],
            [9.8, 57.0],
            [9.6, 57.1]
        ]
    ]
}

response = requests.post(url, headers=headers, json=data)

print("Status Code", response.status_code)
print("Response ", response.json())

print(f"{platform_domain}/api/catalogue/stac/catalogs/user-datasets/catalogs/{workspace}/catalogs/commercial-data/catalogs/planet/catalogs")

Status Code 200
Response  {'message': 'Item ordered successfully'}
https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/user-datasets/catalogs/sgillies-tpzuk/catalogs/commercial-data/catalogs/planet/catalogs


## EO Packages

Users can install EO data packages to view their data

In [4]:
import sys
!{sys.executable} -m pip install --upgrade matplotlib numpy pillow folium

import os

import shapely 
import geopandas as gpd
import folium

import urllib.request
from io import BytesIO 
from PIL import Image

rut_pnt = shapely.Point(-0.683261054299237, 52.672193937442586) # a site near Rutland
thet_pnt = shapely.Point(0.6715892933273722, 52.414471075812315) # a site near Thetford

# Optional cell
# If you want to see these points on a map run this cell

# Create a map (m) centered between the two points
center_lat = (rut_pnt.y + thet_pnt.y) / 2
center_lon = (rut_pnt.x + thet_pnt.x) / 2

m = folium.Map(location=[center_lat, center_lon], zoom_start=8)

# Add markers for each point
folium.Marker([rut_pnt.y, rut_pnt.x], popup="Rutland Site", icon=folium.Icon(color="blue")).add_to(m)
folium.Marker([thet_pnt.y, thet_pnt.x], popup="Thetford Site", icon=folium.Icon(color="green")).add_to(m)

# Step 4: Display the map
m

  Using cached matplotlib-3.10.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached folium-0.19.4-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached contourpy-1.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (101 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
  Using cached branca-0.8.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached xyzservices-2025.1.0-py3-none-any.whl.metadata (4.3 kB)
Using cached matplotlib-3.10.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.6 MB)
Using cach

## PyEODH

A user can use Oxidian's PyEODH client to work with the UK EO DataHub's API.

In [5]:
import pyeodh

# Connect to the Hub
client = pyeodh.Client().get_catalog_service()

# Print a list of the collections held in the Resource Catalogue (their id and description).
# As the Resource Catalogue fills and development continues, the number of collections and the richness of their descriptions will increase
for collect in client.get_collections():
    print(f"{collect.id}: {collect.description}")

ukcp: Regional climate model projections produced as part of the UK Climate Projection 2018 (UKCP18) project. The data produced by the Met Office Hadley Centre provides information on changes in climate for the UK until 2080, downscaled to a high resolution (12km), helping to inform adaptation to a changing climate. The projections cover Europe and a 100 year period, 1981-2080, for a high emissions scenario, RCP8.5. Each projection provides an example of climate variability in a changing climate, which is consistent across climate variables at different times and spatial locations. This dataset contains 12km data for the United Kingdom, the Isle of Man and the Channel Islands provided on the Ordnance Survey's British National Grid.
sentinel2_ard: These data have been created by the Department for Environment, Food and Rural Affairs (Defra) and Joint Nature Conservation Committee (JNCC) in order to cost-effectively provide high quality, Analysis Ready Data (ARD) for a wide range of appl